# Analyze LUCAS data
First of all, let's install `st_lucas` Python package. 

In the case that this notebook is run in [Google Colab](https://colab.research.google.com/), additional steps are needed. Google Colab comes with GDAL 2.2. We have to update this library to version 3.0. It will takes a while.

In [ ]:
!add-apt-repository --yes ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt install libgdal26 libsqlite3-mod-spatialite 
!pip install gdal==3.0.4
print("INSTALLATION COMPLETED")

In [ ]:
!pip install owslib
!pip install -e 'git+https://gitlab.com/geoharmonizer_inea/st_lucas/st_lucas-python-package.git@colab#egg=st_lucas'

Now **we have to restart runtime**: `Runtime -> Restart runtime`.

From `st_lucas` package let's import `LucasRequest` and `LucasIO`.

In [ ]:
from st_lucas import LucasRequest, LucasIO, LucasClassAggregate

#### Define request

Request LUCAS points located in the Czech Republic and surveyed in 2018.

In [ ]:
request = LucasRequest()
request.countries = ['CZ']
request.years = [2018]

#### Download data

In [ ]:
lucasio = LucasIO()
lucasio.download(request)

#### Inspect downloaded LUCAS data

In [ ]:
df = lucasio.to_geopandas()
df[["point_id","lc1"]]

In [ ]:
df.count()

#### Show statistics

In [ ]:
df["lc1"].describe()

In [ ]:
df['lc1_l1'] = df['lc1'].str.slice(0,1)
df['lc1_l1'].value_counts().plot.pie(figsize=(10, 10), autopct="%.1f")

## Class aggregation

LUCAS points are by default obtained with information corresponding to LUCAS land cover level 3. With the `LucasClassAggregate` method, you can get information about level 2 or level 1. New column will be added at the end of attribute table.

In [ ]:
from st_lucas import LucasRequest, LucasIO, LucasClassAggregate
request = LucasRequest()
request.countries = ['CZ']
request.years = [2018]
lucasio = LucasIO()
lucasio.download(request)

lc1_to_level1 = {
    "A00": ["A11", "A12", "A13", "A21", "A22", "A30"],
    "B00": ["B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "B21", "B22", "B23", "B31", "B32", "B33", "B34", "B35", "B36", "B37", "B41", "B42", "B43", "B44", "B45", "B51", "B52", "B53", "B54", "B55", "B71", "B72", "B73", "B74", "B75", "B76", "B77", "B81", "B82", "B83", "B84"],
    "Bx1": ["Bx1"],
    "Bx2": ["Bx2"],
    "C00": ["C10", "C21", "C22", "C23", "C31", "C32", "C33"],
    "D00": ["D10", "D20"],
    "E00": ["E10", "E20", "E30"],
    "F00": ["F10", "F20", "F30", "F40"],
    "G00": ["G11", "G12", "G21", "G22", "G30", "G40", "G50"],
    "H00": ["H11", "H12", "H21", "H22", "H23"]
}

lucasaggr = LucasClassAggregate(lucasio.data, mappings=lc1_to_level1)
lucasaggr.apply()

Check the appearance of the new column.

In [ ]:
df = lucasio.to_geopandas()
df[["lc1_h", "lc1_a"]].head()


## Nomenclature translation

Translate the obtained data into CLC2.

In [ ]:
from st_lucas import LucasClassTranslate

lucastrans = LucasClassTranslate(lucasio.data)
lucastrans.set_translations("CLC2")
lucastrans.apply()

Check the appearance of the new column.

In [ ]:
df = lucasio.to_geopandas()
df[["lc1_h", "lu1_h", "clc2"]].head(10)